# Assignment 2 - Scott Wilkinson (V00887986) 

## Bonus: Creating a pipeline for Q4

A notebook which uses the GridSearch function to obtain both the optimal hyperparameters AND optimal data preparation choices (e.g. n_components for PCA) for a logistic regression model to classify simulated galaxies from Illustris TNG as pre-mergers or post-mergers.

First, we must import the necessary Python packages.

In [1]:
# importing packages used in notebook
import numpy as np
import matplotlib.pyplot as plt
import pymysql, os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# deprecation warnings make the notebook hard to read but do not affect results
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# going full nuclear
warnings.filterwarnings("ignore")

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, 

## Importing Morphology Data

In [2]:
#Query SQL for morphology data
# connects to database
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
# select morphology params that have unflagged morphology fits
x = 'SELECT  dbID, asymmetry, shape_asymmetry, gini_m20_merger, concentration, outer_asymmetry, deviation, multimode, intensity \
    FROM simCFIS_morph\
    WHERE flag_morph = 0\
    AND asymmetry > -1'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()

# save names as a string
names_morph = np.array(db_data, dtype = str).T[0]

# save rest of data as floats
morph = np.array(db_data, dtype = float).T[1:]

print(morph.shape)

(8, 219933)


# Merger Sample Selection

## Selecting a Sample of Pre-Merger Galaxies

Here is the reasoning behind my selection cuts:

1.  snap >= 50 requires z<1 in the simulation (ie. enough time for galaxies to form, evolve and begin to merge)
2. rsep > 0 requires there to be two merging galaxies that are not on top of each other yet
3. mass ratio > 0.1, the merger will be significant when it occurs
4. Flaguntilmerger = 1 ensures the merger is true and not a projection effect that confuses the `subfind` code (see Hani et al. 2020, Rodriguez-Gomez et al. 2015)
5. Tuntilmerger < 0.5 ensures the merger will happen soon (< 500 Myr) and the galaxies have begun to gravitationally interact
6. Tpostmerger > 0.5 ensures no recent has occurred that would cause the disturbed morphology

In [3]:
#Query MySQL with reasoning described above
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'IllustrisTNG100_1', user = 'swilkinson', passwd = '123Sdss!@#')
x ='SELECT e.DB_ID, e.Mstar, e.Tuntilmerger, e.MassRatiountilmerger\
    FROM Environment e \
    WHERE snapNum>=50\
    AND rsep>0 \
    AND MassRatiountilmerger>=0.1\
    AND Flaguntilmerger = 1\
    AND Tuntilmerger < 0.5\
    AND Tpostmerger > 0.5'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()

names_db_pre = np.array(db_data, dtype = str).T[0]
Mstar_pre = np.array(db_data, dtype = float).T[1]
Tum_pre = np.array(db_data, dtype = float).T[2]
mu_pre = np.array(db_data, dtype = float).T[3]

# give time before merger a negative value
Tum_pre *= -1

# differentiate pre-merger galaxies that merge within the next snapshot from post-merger galaxies
#   that have merged within the last snap shot by adding / subtracting 0.1Gyr (approx the middle of a snapshot)
Tum_pre[Tum_pre == 0] = -0.05

# add 0 to match formatting with morphology catalogue
names_db_pre = np.array(['0'+n for n in names_db_pre])

print('There are {} pre-merger galaxies.'.format(len(names_db_pre)))

# match the two catalogues
match, idx_morph_pre, idx_pre = np.intersect1d(names_morph, names_db_pre, return_indices = True)

print('There are {} pre-merger galaxies with flag-free morphology data.'.format(len(names_db_pre[idx_pre])))

There are 6985 pre-merger galaxies.
There are 2783 pre-merger galaxies with flag-free morphology data.


## Selecting a Sample of Post-Merger Galaxies

Here is the reasoning behind my selection cuts:

1. snap >= 50 requires z<1 in the simulation (ie. enough time for galaxies to form, evolve and begin to merge)
2. mass ratio > 0.1, the merger that occurred was significant
3. Flagpostmerger = 1 ensures the merger is true and not a projection effect that confuses the `subfind` code
4. Tpostmerger < 0.5 requires a merger to have occurred in the last 500 Myr
5. rsep > 25 and Tuntilmerger < 0.5 requires there to be no nearby galaxies about to cause another merger in the next 500 Myr

In [4]:
#Query MySQL with the reasoning listed above
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'IllustrisTNG100_1', user = 'swilkinson', passwd = '123Sdss!@#')
x ='SELECT e.DB_ID, e.Mstar, e.Tpostmerger, e.MassRatio\
    FROM Environment e \
    WHERE snapNum>=50\
    AND MassRatio>=0.1\
    AND Flagpostmerger = 1\
    AND Tpostmerger < 0.5\
    AND rsep>25\
    AND Tuntilmerger > 0.5'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()

names_db_post = np.array(db_data, dtype = str).T[0]
Mstar_post = np.array(db_data, dtype = float).T[1]
Tpm_post = np.array(db_data, dtype = float).T[2]
mu_post = np.array(db_data, dtype = float).T[3]

# add 0 to match formatting with morphology catalogue
names_db_post = np.array(['0'+n for n in names_db_post])

# differentiate pre-merger galaxies that merge within the next snapshot from post-merger galaxies
#   that have merged within the last snap shot by adding / subtracting 0.1Gyr (approx the middle of a snapshot)
Tpm_post[Tpm_post == 0] = 0.05

print('There are {} post-merger galaxies.'.format(len(names_db_post)))

# match with morphology catalogue
match, idx_morph_post, idx_post = np.intersect1d(names_morph, names_db_post, return_indices = True)

print('There are {} post-merger galaxies with flag-free morphology data.'.format(len(names_db_post[idx_post])))

names_db_post = names_db_post[idx_post]#[0:len(Tum_pre[idx_pre])]
Mstar_post = Mstar_post[idx_post]#[0:len(Tum_pre[idx_pre])]
Tpm_post = Tpm_post[idx_post]#[0:len(Tum_pre[idx_pre])]
mu_post = mu_post[idx_post]#[0:len(Tum_pre[idx_pre])]

There are 4853 post-merger galaxies.
There are 3192 post-merger galaxies with flag-free morphology data.


# Preparing Data for Model

In [5]:
# list of all merging galaxies
names_mergers = np.array(list(names_db_pre) + list(names_db_post))
T_merger = np.array(list(Tum_pre) + list(Tpm_post))
mu = np.array(list(mu_pre) + list(mu_post))
mass = np.array(list(Mstar_pre) + list(Mstar_post))

# match catalogues
match, idx_morph, idx_merge = np.intersect1d(names_morph, names_mergers, return_indices = True)

T_merger = T_merger[idx_merge]
morph = morph[:,idx_morph]

In [6]:
# shape of output
print(morph.shape)

(8, 5975)


In [7]:
# shape of output
print(T_merger.shape)

(5975,)


## Create Pipeline

In [8]:
# create training and validation sets (75/25 split)
input_tr,input_va,target_tr, target_va = train_test_split(morph.T, T_merger,test_size=0.25, shuffle = True)


target_tr_orig = target_tr.copy()
target_va_orig = target_va.copy()

# prep for classification, ie. change T_merger to binary; 0 if pre-merger, 1 if post-merger
target_tr[target_tr<0] = 0
target_tr[target_tr>0] = 1
target_va[target_va<0] = 0
target_va[target_va>0] = 1

In [9]:
pipe = Pipeline([('scaler', StandardScaler()),\
                 ('pca', PCA()),\
                 ('LR', LogisticRegression())])

In [10]:
pipe.fit(input_tr, target_tr)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('LR', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## We can use the pipeline we've made with gridsearch to easily find the best hyerparameters for the model and best data preparation choices (e.g. n_components for PCA) to solve our problem

In [11]:
# create a grid of parameters to try

# type of penalty term applied
penalty  = ['l1', 'l2']

# Algorithm to use in the optimization problem
solver = ['liblinear', 'saga']

# Tolerance for stopping criteria
tol = [1., 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

# Inverse of regularization strength, smaller values specify stronger regularization.
C = [0.01, 0.1, 0.25, 0.5, 1.0, 1.5 , 2, 10]

# Maximum number of iterations taken for the solvers to converge.
max_iter = [1, 10, 50, 100, 150, 200, 500]

#n_components = [1,5,10,20,30,50,75,100,125,150]
n_components = [1,2,3,4,5,6,7]

#param_grid = [{"penalty": penalty, "solver": solver, "tol": tol, "C": C, "max_iter": max_iter }]
param_grid = [{"pca__n_components": n_components, "LR__penalty": penalty, "LR__solver": solver, "LR__tol": tol, "LR__C": C, "LR__max_iter": max_iter}]


In [12]:
gs = GridSearchCV(pipe, param_grid, cv = 5)

gs.fit(input_tr, target_tr)

print(gs.best_params_)
print(gs.best_score_)

{'LR__C': 0.5, 'LR__max_iter': 1, 'LR__penalty': 'l1', 'LR__solver': 'saga', 'LR__tol': 0.0001, 'pca__n_components': 5}
0.7190359294800268


Most interesting resultis:

1. max_iter = 1? This would mean that multiple iterations only works to worsen the model
2. only uses 5 PCA components, does this mean the rest of the data wasn't just not useful, but actually harmful?